In [13]:
import cv2
import os

In [14]:
def convertImg(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def faceNetLocalize(img):
    img_cvt = convertImg(img)
    return faceNet.detectMultiScale3(img_cvt, scaleFactor=scaleFactor, minNeighbors=minNeighbors, minSize=minSize, outputRejectLevels = True)


In [15]:
scaleFactor = 1.1 #between 1.05 (quality) and 1.4 (speed) recommended (scale of the faces we search for)
minNeighbors = 4 #between 3 (quantity) and 6 (quality) recommended
minSize = (10, 10) #min size of a face in the picture
#maxSize = (240, 240) #max face size in picture

In [16]:
#TODO: testing with different model types
faceNet = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

img = cv2.imread('../img/train/with_mask/0-with-mask.jpg')
#img = cv2.imread('testPic4.jpg')

faceLocs, rejectLevels, confidences = faceNetLocalize(img)
#print(confidences)

In [17]:
crop_imgs = []

for (x, y, w, h) in faceLocs:
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    crop_imgs.append(img[y:y+h, x:x+w])
    

cv2.imshow('img', img)
cv2.waitKey(0)

13

#### Testing - Face detection

In [ ]:
##TODO: finish testing methods, 
# can we even use accuracy methods, should we just test freely with live feed and empirical testing?
#labels_path = os.path.join('..', 'img', 'test', 'labels')

#for img in os.listdir(labels_path):
 #   ...


FileNotFoundError: [WinError 3] The system cannot find the path specified: '..\\img\\test\\labels'

#### Training - Mask classifier

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import matplotlib as plt
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import Sequential
from tensorflow.keras import losses as lfs
from tensorflow.keras.callbacks import ModelCheckpoint

In [20]:
##Dieser ganze Abschnitt ist glaube ich unnötig :thinking:

target_size = (180, 180)

imgs_path = os.path.join('..', 'img')
imgs_path_tr = os.path.join(imgs_path, 'train')
imgs_path_te = os.path.join(imgs_path, 'test')

images = []
labels = []

for label in os.listdir(imgs_path_tr):
    for path_end in os.listdir(os.path.join(imgs_path_tr, label)):
        path = os.path.join(imgs_path_tr, label, path_end)
    
        if os.path.isfile(path):
            img = img_to_array(load_img(path, target_size=target_size))
            images.append(img)
            labels.append(label)

In [21]:
batch_size = 32
img_size = (180, 180)
epochs = 10
checkpoint_path = "mask_model/weights.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

correct_usage=  'correct usage: \n' + 'predict([path to image], \'category\' \n' + 'predict([path to image], \'probabilities\' \n' + 'predict([path to image], \'detection\' \n' + 'predict(\'live_detection\')'

In [22]:
train_ds = image_dataset_from_directory(imgs_path,  validation_split=0.2, subset="training",  seed=123, image_size=img_size,  batch_size=batch_size)
val_ds = image_dataset_from_directory(imgs_path,  validation_split=0.2, subset="validation",  seed=123, image_size=img_size,  batch_size=batch_size)

#train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
#val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 1395 files belonging to 4 classes.
Using 1116 files for training.
Found 1395 files belonging to 4 classes.
Using 279 files for validation.


In [23]:
normalization_layer = Rescaling(1/255)
num_classes = 4

model = Sequential([
    normalization_layer, 
    Conv2D(32, 3, activation='relu'),
    AveragePooling2D(pool_size=(7, 7)),
    Flatten(name="flatten"),
    Dense(128, activation="relu"),
    Dropout(0.5),#drops small confidences
    Dense(num_classes, activation="softmax")
    ])

model.compile(optimizer='adam', loss=lfs.SparseCategoricalCrossentropy, metrics=['accuracy'])


In [24]:
#Callback to save model's weights
cp_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose = 1 )

model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[cp_callback])

Epoch 1/10


TypeError: in user code:

    C:\Users\Julie\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:862 train_function  *
        return step_function(self, iterator)
    C:\Users\Julie\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:852 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Julie\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Julie\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Julie\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Julie\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:845 run_step  **
  

#### Testing (mask): 

In [ ]:
y_test = np.concatenate([y for _, y in val_ds], axis=0)
x_test = np.concatenate([x for x, _ in val_ds], axis=0)
y_pred = model.predict(x_test)
print(classification_report(y_test,y_pred))

##### Predictions:

In [ ]:
#model = Sequential([
#    normalization_layer, 
#    Conv2D(32, 3, activation='relu'),
#    AveragePooling2D(pool_size=(7, 7)),
#    Flatten(name="flatten"),
#    Dense(128, activation="relu"),
#    Dropout(0.5),#drops small confidences
#    Dense(num_classes, activation="softmax")
#    ])
#model.load_weights(checkpoint_path)

In [2]:
def maskPredict(img):
    pred = model.predict([img])
    #TODO: richtiges mapping zwischen index/label?
    label_index = np.argmax(pred)
    return labels[label_index], pred[label_index]


#mode can be 'category', 'probabilities', 'detection', 'live_detection'
def predict(img_path, mode):
    #load_img
    img = ...
    
    if mode=='detection':
        detect(img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return
    if mode=='live_detection':
        live_det()
        return
        
    if mode=='category':
        label, confidence = maskPredict(img)
        return label
        
    if mode=='probabilities':
        return model.predict([img])

    else:
        print(correct_usage)

def predict(mode):
    if mode=='live_detection':
        predict('', mode)
    else:
        print(correct_usage)

def detect(img):
    faceLocs, rejectLevels, confidences = faceNetLocalize(img)
        
    for (x, y, w, h) in faceLocs:
        #crop image and predict label of cropped image
        img_crop = img[y:y+h, x:x+w]
        label, confidence_mask = maskPredict(img_crop)
        #show label/ bounding box on image
        cv2.putText(img, f"{label}, confidence:{confidence_mask}", (x+w-30, y+h), cv2.FONT_HERSHEY_PLAIN, 1.0, cv2.CV_RGB(0,255,0), 2.0) 
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    cv2.imshow('live_output', img)

def live_det():
    #TODO: errorhandling for camera
    
    wait_time = 20 #time in ms to wait before refreshing feed
    camera = cv2.Videcapture(0) #Input value might differ on different systems
    
    while(True):
        _, img = camera.read()

        detect(img)

        #wait for ESC or q
        if (cv2.waitKey(wait_time) & 0xFF) in [27, ord('q')]: 
            break


    camera.release()
    return 'live_output'


SyntaxError: EOL while scanning string literal (Temp/ipykernel_47988/2690488841.py, line 14)